In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from m3util.viz.arrows import DrawArrow
from m3util.viz.layout import layout_subfigures_inches
from m3util.viz.printing import printer
import matplotlib.pyplot as plt
from m3util.viz.style import set_style
from m3util.viz.images import display_image

import matplotlib.image as mpimg
from belearn.dataset.dataset import BE_Dataset
from m3util.util.IO import download_and_unzip
from belearn.viz.viz import Viz

set_style("printing")

printing = printer(basepath="./Figures/")

In [ ]:
# Download the data file from Zenodo
url = 'https://zenodo.org/record/7774788/files/PZT_2080_raw_data.h5?download=1'

# Specify the filename and the path to save the file
filename = '/data_raw.h5'
save_path = './Data'

# download the file
download_and_unzip(filename, url, save_path)

In [ ]:
data_path = save_path + '/' + filename

# instantiate the dataset object
dataset = BE_Dataset(data_path)

# print the contents of the file
dataset.print_be_tree()

In [ ]:
import numpy as np

# insatiate the visualization object
image_scalebar = [2000, 500, "nm", "br"]

BE_viz = Viz(
    dataset,
    printing,
    verbose=True,
    SHO_ranges=[(0, 1.5e-4), (1.31e6, 1.33e6), (-300, 300), (-np.pi, np.pi)],
    image_scalebar=image_scalebar,
)


In [ ]:
prediction = {"resampled": False, "label": "Raw"}


BE_viz.raw_data_comparison(prediction, pixel=3000, voltage_step=60, filename="Figure_2_1_raw_cantilever_response")


In [ ]:
from m3util.viz.arrows import DrawArrow
from m3util.viz.layout import layout_subfigures_inches
import matplotlib.pyplot as plt
from m3util.viz.style import set_style


pixel = 1500

# Dictionary defining the subfigures with positions in inches
subfigures_dict = {
    "AFM_Image": {
        "position": (0, 6.5 - 3.2131, 2.0972 - 18 * 3 / 72, 2.0972 - 18 * 3 / 72),
        "skip_margin": True,
    },
    "real_imag": {"position": (2.2014 - (18 * 3) / 72, 6.5 - 2.1364, 2.0972, 2.0972)},
    "amp_phase": {"position": (4.3588 - 18/72, 6.5 - 2.1364, 2.0972, 2.0972)},
    "BE": {"position": (2.2014 - (18 * 2) / 72, 6.5 - 4.2942, 2.0972, 2.0972)},
    "FFT": {"position": (4.3588 - (18) / 72, 6.5 - 4.2942, 2.0972, 2.0972)},
    "Bipolar": {"position": (0.0532, 6.5 - 6.4609, 4.2361, 2.0972)},
    "Hysteresis": {"position": (4.3588, 6.5 - 6.4609, 2.0972, 2.0972)},
}



# Create a custom figure of size (10 inches width, 6 inches height)
fig, ax = layout_subfigures_inches(size=(6.5, 6.5), subfigures_dict=subfigures_dict)

arrow = DrawArrow(
    fig,
    start_pos=(1.0926 - 3 * 18 / 72, 6.5 - 1 - 3* 18 / 72),
    end_pos=(2.2014 - 3 * 18 / 72, 6.5 - 0.4972 - 3 * 18 / 72),
    text="FFT",
    text_position="center",
    text_alignment="center",
    vertical_text_displacement="top",
)

display_image(ax["AFM_Image"], "assets/AFM_field.png")

arrow.draw()

true = {"resampled": False, "label": "Raw"}

BE_viz.plot_real_imainary(ax["real_imag"], true, pixel=3000, voltage_step=60, 
                          add_arrows={'imag_value':1.36e6, 'real_value':1.26e6, 'height':0.15, 'width':0.05})


BE_viz.plot_magnitude_spectrum(ax["amp_phase"], true, pixel=3000, voltage_step=60)

ax["BE"].plot([0, 1], [0, 1], "k--", lw=0.5)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

def get_closest_point(x_data, y_data, value, axis='x'):
    """
    Get the closest point on a line plot to a provided x or y value.

    Parameters
    ----------
    x_data : array-like
        Array of x data points.
    y_data : array-like
        Array of y data points.
    value : float
        The x or y value to find the closest point to.
    axis : {'x', 'y'}, optional
        Specify which axis to use for finding the closest point (default is 'x').

    Returns
    -------
    tuple
        (closest_x, closest_y): The closest point on the line plot.

    Raises
    ------
    ValueError
        If the axis is not 'x' or 'y', or if x_data and y_data have different lengths.
    """
    # Ensure x_data and y_data are NumPy arrays
    x_data = np.asarray(x_data)
    y_data = np.asarray(y_data)

    # Check that x_data and y_data have the same length
    if x_data.shape != y_data.shape:
        raise ValueError("x_data and y_data must have the same shape.")

    # Find the index of the closest point
    if axis == 'x':
        idx = np.abs(x_data - value).argmin()
    elif axis == 'y':
        idx = np.abs(y_data - value).argmin()
    else:
        raise ValueError("axis must be 'x' or 'y'")

    return x_data[idx], y_data[idx]

def draw_ellipse_with_arrow_on_line(ax, x_data, y_data, value, width, height, axis='x',
                                    line_direction='horizontal', arrow_position='top',
                                    arrow_length_frac=0.3, color='blue', linewidth=2,
                                    arrow_props=None, ellipse_props=None):
    """
    Draw an ellipse with an arrow at a specific point on a line plot.

    Parameters
    ----------
    ax : matplotlib.axes.Axes
        Matplotlib axis where the ellipse and arrow will be drawn.
    x_data : array-like
        X data points of the line plot.
    y_data : array-like
        Y data points of the line plot.
    value : float
        The x or y value at which to place the ellipse.
    width : float
        Width of the ellipse as a fraction of the x-axis range.
    height : float
        Height of the ellipse as a fraction of the y-axis range.
    axis : {'x', 'y'}, optional
        Axis to find the closest point on (default is 'x').
    line_direction : {'horizontal', 'vertical'}, optional
        Direction of the line to which the ellipse and arrow are related (default is 'horizontal').
    arrow_position : {'top', 'bottom'}, optional
        Position to place the arrow relative to the ellipse (default is 'top').
    arrow_length_frac : float, optional
        Length of the arrow as a fraction of the axis range (default is 0.3).
    color : str, optional
        Color of the ellipse and arrow (default is 'blue').
    linewidth : float, optional
        Line width of the ellipse (default is 2).
    arrow_props : dict, optional
        Additional properties to customize the arrow appearance.
    ellipse_props : dict, optional
        Additional properties to customize the ellipse appearance.

    Raises
    ------
    ValueError
        If invalid values are provided for axis, line_direction, or arrow_position.
    """
    # Ensure x_data and y_data are NumPy arrays
    x_data = np.asarray(x_data)
    y_data = np.asarray(y_data)

    # Check that x_data and y_data have the same length
    if x_data.shape != y_data.shape:
        raise ValueError("x_data and y_data must have the same shape.")

    # Get the closest point on the line plot
    ellipse_center = get_closest_point(x_data, y_data, value, axis=axis)

    # Get axis limits for scaling dimensions and arrow length
    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()

    # Calculate arrow length based on line direction
    if line_direction == 'horizontal':
        arrow_length = arrow_length_frac * (x_max - x_min)
    elif line_direction == 'vertical':
        arrow_length = arrow_length_frac * (y_max - y_min)
    else:
        raise ValueError("line_direction must be 'horizontal' or 'vertical'")

    # Scale the width and height of the ellipse
    width_scaled = width * (x_max - x_min)
    height_scaled = height * (y_max - y_min)

    # Set default properties for the ellipse and update with any additional properties
    default_ellipse_props = {'edgecolor': color, 'facecolor': 'none', 'lw': linewidth}
    if ellipse_props:
        default_ellipse_props.update(ellipse_props)

    # Draw the ellipse
    ellipse = Ellipse(xy=ellipse_center, width=width_scaled, height=height_scaled, **default_ellipse_props)
    ax.add_patch(ellipse)

    # Calculate the start and end points of the arrow based on position and direction
    if line_direction == 'horizontal':
        if arrow_position == 'top':
            start_point = (ellipse_center[0], ellipse_center[1] + height_scaled / 2)
            end_point = (ellipse_center[0] + arrow_length, ellipse_center[1] + height_scaled / 2)
        elif arrow_position == 'bottom':
            start_point = (ellipse_center[0], ellipse_center[1] - height_scaled / 2)
            end_point = (ellipse_center[0] + arrow_length, ellipse_center[1] - height_scaled / 2)
        else:
            raise ValueError("arrow_position must be 'top' or 'bottom'")
    elif line_direction == 'vertical':
        if arrow_position == 'top':
            start_point = (ellipse_center[0] + width_scaled / 2, ellipse_center[1])
            end_point = (ellipse_center[0] + width_scaled / 2, ellipse_center[1] + arrow_length)
        elif arrow_position == 'bottom':
            start_point = (ellipse_center[0] - width_scaled / 2, ellipse_center[1])
            end_point = (ellipse_center[0] - width_scaled / 2, ellipse_center[1] + arrow_length)
        else:
            raise ValueError("arrow_position must be 'top' or 'bottom'")
    else:
        raise ValueError("line_direction must be 'horizontal' or 'vertical'")

    # Set default properties for the arrow and update with any additional properties
    default_arrow_props = {'facecolor': color, 'width': 2, 'headwidth': 10, 'headlength': 10, 'linewidth': 0}
    if arrow_props:
        default_arrow_props.update(arrow_props)

    # Draw the arrow
    ax.annotate('', xy=end_point, xytext=start_point, arrowprops=default_arrow_props)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Sample data
x = np.linspace(0, 10, 100)
y = np.sin(x)

# Create a plot
fig, ax = plt.subplots()
ax.plot(x, y, label="Sine Wave")

# Draw an ellipse with an arrow on the plot
draw_ellipse_with_arrow_on_line(
    ax,
    x_data=x,
    y_data=y,
    value=4,
    width=0.1,           # Fraction of the x-axis range
    height=0.1,          # Fraction of the y-axis range
    axis='x',
    line_direction='vertical',
    arrow_position="top",
    arrow_length_frac=0.2,
    color='red',
    linewidth=2,
    arrow_props={'facecolor': 'red', 'width': 2, 'linewidth': 0},
    ellipse_props={'linestyle': '--'}
)

# Customize and display the plot
ax.grid(True)
ax.legend()
plt.show()
